In [1]:
import os
import glob
import re
import pandas as pd
import numpy as np

import tensorflow as tf
from tensorflow import keras
from tensorflow.train import FloatList, Int64List, BytesList, Feature, Features, FeatureList, FeatureLists, Example
from tensorflow.io import FixedLenSequenceFeature

import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()

import soundfile as sf

try:
    os.environ['KAGGLE_DATA_PROXY_TOKEN']
except KeyError:
    dir_out = "./"
    dir_files = "Respiratory_Sound_Database/Respiratory_Sound_Database/"
else:
    dir_out = "/kaggle/working/"
    dir_files = "/kaggle/input/respiratory-sound-database/Respiratory_Sound_Database/Respiratory_Sound_Database/"

dir_audio = dir_files + "audio_and_txt_files/"
fname_tfrecord = dir_out + "wavs_" # will be segmented files; _0, _1, etc.

In [2]:
### Read WAVs and pad ###
    
group_pat_num = "([0-9]{3})"
group_rec_index = "([0-9][a-z][0-9])"
group_chest_loc = "(Tc|Al|Ar|Pl|Pr|Ll|Lr)"
group_acc_modes = "(sc|mc)"
group_equipments = "(AKGC417L|LittC2SE|Litt3200|Meditron)"

regex_info = re.compile("_".join([group_pat_num, group_rec_index, group_chest_loc, group_acc_modes, group_equipments]))

top = os.getcwd()
os.chdir(dir_audio)
fnames = glob.glob("*.wav")

l_wav_rec = []
dict_wav_rec = {}
min_len = np.inf
max_len = 0

feature = {}
unique_rec_ids = []

### !!!!!!!!!!!! remove [:3]

for fname_ix in range(len(fnames)):
    fname = fnames[fname_ix]
    match_info = regex_info.match(fname)
    pat_num = int(match_info.group(1))
    rec_index = match_info.group(2)
    chest_loc = match_info.group(3)
    acc_mode = match_info.group(4)
    equipment = match_info.group(5)
    
    wav_content = sf.read(fname)[0]
    l_wav_rec.append([pat_num, rec_index, chest_loc, wav_content])
    dict_wav_rec[(pat_num, rec_index, chest_loc)] = wav_content
    
    unique_rec_id = "{} {} {}".format(pat_num, rec_index, chest_loc)
    unique_rec_ids.append(unique_rec_id)
    
    f = Feature(
        float_list = FloatList(value=wav_content)
    )
    
    feature[unique_rec_id] = f
    
    if (fname_ix + 1) % 50 == 0: # + 1 only so it doesn't write immediately at ix 0
        features = Features(feature = feature)
        example = Example(features = features)

        tfr_options = tf.io.TFRecordOptions(compression_type="GZIP")
        
        f_num_suffix = fname_ix // 50

        with tf.io.TFRecordWriter("../../../tfrecords/{}{}.tfrecord".format(fname_tfrecord, f_num_suffix),
                                  tfr_options) as f:
            f.write(example.SerializeToString())
        
        feature = {}
    
#     if len(wav_content) > max_len:
#         max_len = len(wav_content)
#         # for getting the corresponding annotation below
#         max_patnum = pat_num
#         max_recindex = rec_index
#         max_chestloc = chest_loc
    
#     if len(wav_content) < min_len:
#         min_len = len(wav_content)
#         # for getting the corresponding annotation below
#         min_patnum = pat_num
#         min_recindex = rec_index
#         min_chestloc = chest_loc

features = Features(feature = feature)
example = Example(features = features)

tfr_options = tf.io.TFRecordOptions(compression_type="GZIP")

with tf.io.TFRecordWriter("../../../tfrecords/{}{}.tfrecord".format(fname_tfrecord, f_num_suffix + 1),
                          tfr_options) as f:
    f.write(example.SerializeToString())

os.chdir(top)

# pad all recordings to same length
# for i in range(len(l_wav_rec)):
#     if len(l_wav_rec[i][3]) < max_len:
#         padding = [0] * ( max_len - len(l_wav_rec[i][3]) )
#         l_wav_rec[i][3] = np.append(l_wav_rec[i][3], padding)

# pad all recordings to multiple of length of shortest recording
# for i in range(len(l_wav_rec)):
#     if len(l_wav_rec[i][3]) % min_len != 0:
#         padding = [0] * ( min_len - len(l_wav_rec[i][3]) % min_len)
#         l_wav_rec[i][3] = np.append(l_wav_rec[i][3], padding)

# l_wav_rec.sort(key=lambda subl: (subl[0], subl[1], subl[2]))

# wav_cols = ["Patient number", "Recording index", "Chest location", "WAV"]
# df_wav_rec = pd.DataFrame(l_wav_rec, columns=wav_cols)

######

# for wav_rec in l_wav_rec:
#     wav_transposed = tf.reshape(tf.constant(l_wav_rec[0][3], dtype=tf.float32), shape=[-1, 1])

feat_patnum = Feature(
    int64_list = Int64List(value=[l_wav_rec[0][0]])
)

feat_recix = Feature(
    bytes_list = BytesList(value=[bytes(l_wav_rec[0][1], "ascii")])
)

feat_chestloc = Feature(
    bytes_list = BytesList(value=[bytes(l_wav_rec[0][2], "ascii")])
)

features_context = Features(
    feature = {
        "patient number" : feat_patnum,
        "recording index" : feat_recix,
        "chest location" : feat_chestloc
    }
)

# wav_transposed = tf.reshape(tf.constant(l_wav_rec[0][3], dtype=tf.float32), shape=[-1, 1])

# fl = []

# for val_ix in range(len(l_wav_rec[0][3])):
#     feature = Feature(
#         float_list = FloatList(value=([l_wav_rec[0][3][val_ix]]))
#     )
#     fl.append(feature)

# feature = Feature(
#     float_list = FloatList(value=l_wav_rec[0][3])
# )

# fl.append(feature)

# feat_list = FeatureList(feature = fl)

# fls = {}    
# fls["{} {} {}".format(l_wav_rec[0][0], l_wav_rec[0][1], l_wav_rec[0][2])] = feat_list

# feat_lists = FeatureLists(feature_list = fls)

# se = SequenceExample(
#     context = features_context,
#     feature_lists = feat_lists
# )

In [16]:
df = pd.DataFrame(unique_rec_ids)
df.to_csv("unique_rec_ids.csv", index=False)

In [17]:
df = pd.read_csv("unique_rec_ids.csv")

In [24]:
# feature_description = {
#         "patient number" : tf.io.FixedLenFeature([], tf.int64),
#         "recording index" : tf.io.FixedLenFeature([], tf.string),
#         "chest location" : tf.io.FixedLenFeature([], tf.string)
#     }

# seq_description = {
#         "{} {} {}".format(l_wav_rec[0][0], l_wav_rec[0][1], l_wav_rec[0][2]) : FixedLenSequenceFeature(max_len, float),
#     }

# for serialized in tf.data.TFRecordDataset([fname_tfrecord], compression_type="GZIP"):
#     parsed = tf.io.parse_sequence_example(serialized, feature_description, seq_description)
#     print(parsed)


# feat_parse_desc = dict(zip(unique_rec_ids, [tf.io.VarLenFeature(tf.float32)] * len(unique_rec_ids)))
feat_parse_desc = dict(zip(df["0"], [tf.io.VarLenFeature(tf.float32)] * len(df)))

for serialized in tf.data.TFRecordDataset(["./tfrecords/wavs_0.tfrecord"], compression_type="GZIP"):
    parsed = tf.io.parse_example(serialized, feat_parse_desc)
parsed

{'101 1b1 Al': <tensorflow.python.framework.sparse_tensor.SparseTensor at 0x14a90d4edf0>,
 '101 1b1 Pr': <tensorflow.python.framework.sparse_tensor.SparseTensor at 0x14a90d4efd0>,
 '102 1b1 Ar': <tensorflow.python.framework.sparse_tensor.SparseTensor at 0x14a90d4ed00>,
 '103 2b2 Ar': <tensorflow.python.framework.sparse_tensor.SparseTensor at 0x14a90d4ec40>,
 '104 1b1 Al': <tensorflow.python.framework.sparse_tensor.SparseTensor at 0x14a90d4e880>,
 '104 1b1 Ar': <tensorflow.python.framework.sparse_tensor.SparseTensor at 0x14a90d4e670>,
 '104 1b1 Ll': <tensorflow.python.framework.sparse_tensor.SparseTensor at 0x14a90d4e460>,
 '104 1b1 Lr': <tensorflow.python.framework.sparse_tensor.SparseTensor at 0x14a90d4e3d0>,
 '104 1b1 Pl': <tensorflow.python.framework.sparse_tensor.SparseTensor at 0x1499f328dc0>,
 '104 1b1 Pr': <tensorflow.python.framework.sparse_tensor.SparseTensor at 0x1499f328ca0>,
 '105 1b1 Tc': <tensorflow.python.framework.sparse_tensor.SparseTensor at 0x14914ea8760>,
 '106 2b1 

In [ ]:
wav_transposed = tf.reshape(tf.constant(l_wav_rec[0][3], dtype=tf.float32), shape=[-1, 1])
wav_transposed